In [11]:
%pip install graphviz
from graphviz import Digraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 KB 559.4 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
class edge:
      def __init__(self):
        self.label: str = None
        self.destination = None
      def __str__(self):
        return f"edge {self.label} going to {self.destination}"

In [5]:
class NFA_state:
  def __init__(self):
    self.label: str = None
    self.out_edges: list[edge] = []

  def __str__(self) -> str:
    return self.label
  
  def __repr__(self) -> str:
     return self.label
  
  def __eq__(self, other) -> bool:
    return other.label == self.label
  
  def __ne__(self, other) -> bool:
    return other.label != self.label
  
  def __hash__(self) -> int:
     return hash(self.label)

In [6]:
class NFA:

    def __init__(self, initial, acc, inner):
        self.start: NFA_state = initial
        self.accept: NFA_state = acc
        self.inner_states: list[NFA_state] = inner

    
    def __repr__(self) ->str:
      return self.start.label


In [8]:
class SuperState:
  '''A superState is defined by a bunch of SubStates, a bunch of outgoing edges, and its label/ID,
    and 2 flags indicating whether or not this SuperState is starting or accepting
  '''
  def __init__(self):
      self.is_start: bool = False          
      self.is_end: bool = False            
      self.out_edges: list[edge] = []        
      self.sub_states: set[SuperState] = set()        
      self.label: str = None

  def get_labels(self) -> list[str]:
    labels = []
    for s in self.sub_states:
      labels.append(s.label)
    return labels
    
  '''
    gets the direct neighbours from taking the Character input edge, just the direct neighbours.
  '''
  def get_char_closure(self, char) -> set:  # set[SuperState]
    closure = set()
    for edge in self.out_edges:
      if edge.label == char:
        closure.add(edge.destination)
    return closure
  
  '''
    Returns a set of States reachable from taking Char edges recursively
  '''
  def get_reachable_states(self, char) -> set:  # set[SuperState]
    reachable_states = set()
    for edge in self.out_edges:
      if edge.label == char:
        reachable_states.add(edge.destination)
        reachable_states = reachable_states.union(edge.destination.get_reachable_states(char))
    return reachable_states
  
  '''
    Returns a set of States reachable from taking Epsilon edges recursively
  '''
  def get_epsilon_closure(self) -> set:  # set[SuperState]
    epsilon_closure = set()
    epsilon_closure.add(self)
    for edge in self.out_edges:
      if edge.label == "ε":
        epsilon_closure.add(edge.destination)
        epsilon_closure = epsilon_closure.union(edge.destination.get_epsilon_closure())
    return epsilon_closure
    
  def generate_new_superstate(self, char):
      reachable_states = set()
      for state in self.sub_states:
          reachable_states = reachable_states.union(state.get_char_closure(char))

      new_superstate = set()
      for state in reachable_states:
          new_superstate = new_superstate.union(state.get_epsilon_closure())


      # create a new superstate with the new states
      new_super = SuperState()
      new_super.sub_states = new_superstate
      return new_super

  def __eq__(self, other):
    set_of_src_labels = set()
    set_of_dst_labels = set()
    for s in self.sub_states:
      set_of_src_labels.add(s.label)
    for s in other.sub_states:
      set_of_dst_labels.add(s.label)
    return set_of_src_labels == set_of_dst_labels
  
  def __ne__(self, other):
    set_of_src_labels = set()
    set_of_dst_labels = set()
    for s in self.sub_states:
      set_of_src_labels.add(s.label)
    for s in other.sub_states:
      set_of_dst_labels.add(s.label)
    return set_of_src_labels != set_of_dst_labels

  def __str__(self):
    if(self.sub_states):
      stringy = ""
      for sub in self.sub_states:
        stringy = stringy + str(sub) + ","
      return "SuperState " +stringy
    else:
      return "Empty SuperState"

  def __repr__(self):
    if(self.sub_states):
      stringy = ""
      for sub in self.sub_states:
        stringy = stringy + str(sub) + ","
      return stringy
    else:
      return "Empty SuperState"

  def __hash__(self):
    if(self.sub_states):
      stringy = " "
      for sub in self.sub_states:
        stringy = stringy + str(sub) + ","
      return hash("SuperState" +stringy)
    else:
      return hash(None)

In [10]:
class DFA:
    ''' A DFA object contains a set of SuperStates, also a pointer to the starting SuperState and a set of the accept SuperState'''
    def __init__(self):
        self.start_super_state: SuperState = None          
        self.super_states: set[SuperState] = set() 
        self.accept_super_states: set[SuperState] = set()

    
                      

    '''Just makes a new empty DFA, and a legit SuperState then insert this initial SuperState into the Empty DFA then return the DFA'''
    def __init__(self, nfa: NFA):
      self.start_super_state = SuperState()
      self.start_super_state.is_start = True
      self.start_super_state.label = "S0"
      
      self.start_super_state.sub_states = nfa.start.get_epsilon_closure()
      self.start_super_state.out_edges = []
      self.super_states.add(self.start_super_state)
      self.accept_super_states = set()

      
    '''takes a list of states, returns the matching SuperState if the list already'''
    def get_super_state(self, super_state: SuperState):
      for ss in self.super_states:
        if ss == super_state:
          return ss
      return None

    def __eq__(self, other):
      return self.start_super_state == other.start_super_state and self.super_states == self.super_states
    
    def __ne__(self, other):
      return self.start_super_state != other.start_super_state or self.super_states != self.super_states

    
    def empt(self):
      self.start_super_state = SuperState()          
      self.end_super_state = set()             
      self.super_states = set()

    def visualize(self):
      gra = Digraph(graph_attr={'rankdir':'LR', "label":"DFA"})
      id = 0
      for SSTT in self.super_states:
        if(SSTT.is_start):
          gra.node("", _attributes={'shape' : 'none'})
          gra.edge("", SSTT.label)
        if(SSTT.is_end):
          gra.node(SSTT.label, _attributes={'peripheries' : '2'})
        else:
          gra.node(SSTT.label)
        id = id + 1
        
      for SSTT in self.super_states:
        for edg in SSTT.out_edges:
          gra.edge(SSTT.label , edg.destination.label, label=edg.label)

      gra.format = 'png'
      gra.render('DFA', view= True)
      return gra.source
       


def powerset_construction(nfa: NFA, alphabet: str) -> DFA:
    dfa = DFA(nfa)
    unmarked_states = set()
    unmarked_states.add(dfa.start_super_state)

    while unmarked_states:
        current_super_state:SuperState = unmarked_states.pop()
        for char in alphabet:
            generated_super_state = current_super_state.generate_new_superstate(char)
            if generated_super_state:
                existing_super_state = dfa.get_super_state(generated_super_state)
                if not existing_super_state:
                    new_super_state = SuperState()
                    new_super_state.sub_states = generated_super_state.sub_states
                    new_super_state.label = "S" + str(len(dfa.super_states))
                    dfa.super_states.add(new_super_state)
                    unmarked_states.add(new_super_state)
                    current_super_state.out_edges.append(edge(char, new_super_state)) 
                else:
                    current_super_state.out_edges.append(edge(char, existing_super_state))

    for super_state in dfa.super_states:
        for sub_state in super_state.sub_states:
            if sub_state == nfa.accept:
                super_state.is_end = True
                dfa.accept_super_states.add(super_state)

    return dfa
    
    